# Inference from any pose

With this notebook you can run inference of ContourCraft using an arbitrary non-SMPL(-x) mesh sequence.

To run in you will need to have a mesh sequence as a `.pkl` file containing a dictionary of the following items (see `DEFAULTS.data_root/fromanypose/mesh_sequence.pkl` as an example):
* `verts`: np.array [N, V, 3]
* `faces`: np.array [F, 3]

You also need to have a garment mesh (as `.obj` file) which is aligned with the first frame of your body sequence.

In this notebook we first show how to convert a garment mesh in `.obj` format into a garment dict used in HOOD. 

Then, we show how to use the configuration file `aux/from_any_pose.yaml` to run inference with over albitrary SMPL sequence and arbitrary mesh sequence.

## Create template file from the .obj file:

Use `utils.mesh_creation::obj2template()` function to convert an `.obj` file into a template dictionary and then save it with `pickle_dump`

`DEFAULTS.data_root/fromanypose/skirt.obj` is provided as an example

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from utils.mesh_creation import obj2template
from utils.io import pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path

/data/agrigorev/miniforge3/envs/ccraft/lib/python3.10/site-packages/trimesh/geometry.py:7: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.1)
  import scipy.sparse


In [10]:
obj_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'skirt.obj'
out_template_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'skirt.pkl'

template_dict = obj2template(obj_path, verbose=True)
pickle_dump(template_dict, out_template_path)

Adding coarse edges... (may take a while)
Done.


In [11]:
from utils.mesh_creation import add_pinned_verts


pinned_indices = \
[2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215,
 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235,
 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2327, 2328, 2351, 2492, 2497, 2669, 2670, 2671, 2674, 2837, 3139,
 3200, 3204, 3359, 3362, 3495, 3512, 3634, 3638, 3805, 3965, 3967, 4133, 4137, 4140, 4335, 4340, 4506, 4509, 4669, 4674,
 4749, 4812, 4849, 4853, 5138, 5309, 5342, 5469, 5474, 5503, 5646, 5650, 5654, 5855, 5857, 6028, 6091, 6204, 6209, 6280,
 6374, 6377, 6378, 6473, 6649, 6654, 6814, 6817, 6986, 6989, 6990, 6992, 7172, 7178, 7336, 7500, 7505, 7661, 7665, 7666]

add_pinned_verts(out_template_path, pinned_indices)

## Inference with a mesh sequence

Here we use a sequence of arbitrary meshes.  The garment template needs to be aligned with the first frame of the sequence.

The mesh sequence has to be a `.pkl` file containing a dictionary of the following items (see `DEFAULTS.data_root/fromanypose/mesh_sequence.pkl` as an example):
* `verts`: np.array [N, V, 3]
* `faces`: np.array [F, 3]

### Load runner and dataloader from `from_any_pose` config

In [3]:
from utils.arguments import load_params, create_modules
from utils.arguments import load_params
from utils.common import move2device
from utils.io import pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path
import torch
from utils.arguments import create_runner

In [4]:
# use HOOD
# modules, config = load_params('aux/from_any_pose_hood')
# checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'hood_final.pth'

# use ContourCraft
modules, config = load_params('aux/from_any_pose')
checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'contourcraft.pth'


runner_module, runner, aux_modules = create_runner(modules, config)

state_dict =  torch.load(checkpoint_path)
runner.load_state_dict(state_dict['training_module'])

Warp 1.5.1 initialized:
   CUDA Toolkit 12.6, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Quadro RTX 6000" (24 GiB, sm_75, mempool enabled)
     "cuda:1"   : "Quadro RTX 6000" (24 GiB, sm_75, mempool enabled)
     "cuda:2"   : "Quadro RTX 6000" (24 GiB, sm_75, mempool enabled)
     "cuda:3"   : "Quadro RTX 6000" (24 GiB, sm_75, mempool enabled)
     "cuda:4"   : "Quadro RTX 6000" (24 GiB, sm_75, mempool enabled)
     "cuda:5"   : "Quadro RTX 6000" (24 GiB, sm_75, mempool enabled)
     "cuda:6"   : "Quadro RTX 6000" (24 GiB, sm_75, mempool enabled)
     "cuda:7"   : "Quadro RTX 6000" (24 GiB, sm_75, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /local/home/agrigorev/.cache/warp/1.5.1


/tmp/ipykernel_2512564/3623870893.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict =  torch.load(checkpoint_path)


<All keys matched successfully>

### Create a dataloader

Same as in the previous part, set several variables to specify which garment and which pose sequence you want to use:


- `pose_sequence_path` path to the smpl pose sequence relative to `DEFAULTS.data_root`. For this example we use `fromanypose/mesh_sequence.pkl`
- `garment_template_path` path to the .pkl file with the garment template  For this example we use `fromanypose/skirt.pkl`

In [5]:
from utils.datasets import make_fromanypose_dataloader

pose_sequence_path = Path('fromanypose') / 'mesh_sequence.pkl'
garment_template_path = Path('fromanypose') / 'skirt.pkl'


dataloader = make_fromanypose_dataloader(pose_sequence_type='mesh', 
                       pose_sequence_path=pose_sequence_path, 
                       garment_template_path=garment_template_path)

### load sample, infer and save trajectories

To visualise the saved trajectories, see `Inference.ipynb::write_video`

In [6]:
sample = next(iter(dataloader))

sample = move2device(sample, 'cuda:0')
runner.to('cuda:0')
trajectories_dict = runner.valid_rollout(sample)


# Save the sequence to disk
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_m.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)

/local/home/agrigorev/Workdir/contourcraft_private/datasets/from_any_pose.py:252: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647382455/work/aten/src/ATen/native/Cross.cpp:62.)
  normals = torch.cross(v0v1, v0v2)  # (F, 3)
  0%|          | 0/384 [00:00<?, ?it/s]

Module utils.warp_u.proximity b91666f load on device 'cuda:0' took 0.75 ms  (cached)


100%|██████████| 384/384 [01:45<00:00,  3.64it/s]

Rollout saved into /data/agrigorev/02_Projects/ccraft_data/temp/output_m.pkl
